In [1]:
from datetime import datetime
from itertools import combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.prisma import prisma

ModuleNotFoundError: No module named 'src'

In [ ]:
async def startup():
    await prisma.connect()

In [ ]:
async def shutdown():
    await prisma.disconnect()

In [ ]:
await startup()

In [ ]:
def convert_str_timestamp_to_timestamp(timestamp_str: str, timestamp_format: str = "%Y-%m-%dT%H:%M:%S.%fZ") -> int:
    timestamp = datetime.timestamp(datetime.strptime(timestamp_str, timestamp_format))

    return timestamp

In [ ]:
async def get_player_damaged_player_event(event_id: str) -> dict:
    player_damage_player_raw_data = await prisma.playerdamagedplayerevent.find_first(
        where={
            'eventId': event_id,
        }
    )

    player_damage_player_data = player_damage_player_raw_data.dict()

    return player_damage_player_data

In [ ]:
async def get_player_killed_player_event(event_id: str) -> dict:
    player_killed_player_raw_data = await prisma.playerkilledplayerevent.find_first(
        where={
            'eventId': event_id,
        }
    )

    player_killed_player_data = player_killed_player_raw_data.dict()

    return player_killed_player_data

In [ ]:
async def get_rounds_by_match_id(match_id: str) -> list:
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'events': True
        },
    )

    match_rounds_data = []
    for match_round_raw_data in match_rounds_raw_data:
        match_rounds_data.append(match_round_raw_data.dict())

    return match_rounds_data

In [ ]:
async def get_rounds_pdp_pkp_data_by_match_id(match_id: str) -> dict:
    rounds_by_match_id = await get_rounds_by_match_id(match_id)

    # rounds_data = {
    #     round_id: {
    #         pdp: {
    #             actor: {
    #                 target: [(d_d, t_d),...]
    #             }
    #         }
    #         pkp: {
    #             actor: {
    #                 target: (true, t_k)
    #             }
    #         }
    #     }
    # }
    rounds_data = {}
    for round in rounds_by_match_id:
        round_id = round["roundId"]
    
        events = {
            "pdp": {},
            "pkp": {},
        }
        for event in round["events"]:
            event_id = event["eventId"]
            timestamp = event["timestamp"]
            
            if event["eventType"] == "PLAYER_DAMAGED_PLAYER":
                player_damaged_player_event = await get_player_damaged_player_event(event_id)
                
                actor = player_damaged_player_event["actorPlayerId"]
                target = player_damaged_player_event["targetPlayerId"]
                damage = player_damaged_player_event["damageDealt"]
    
                if actor in events["pdp"]:
                    if target in events["pdp"][actor]:
                        events["pdp"][actor][target] += [(damage, timestamp)]
                    else:
                        events["pdp"][actor] = {
                            target: [(damage, timestamp)]
                        }
                else:
                    events["pdp"][actor] = {
                        target: [(damage, timestamp)]
                    }
            elif event["eventType"] == "PLAYER_KILLED_PLAYER":
                player_killed_player_event = await get_player_killed_player_event(event_id)
                
                actor = player_killed_player_event["actorPlayerId"]
                target = player_killed_player_event["targetPlayerId"]
    
                if actor in events["pkp"]:
                    if target in events["pkp"][actor]:
                        continue
                    else:
                        events["pkp"][actor][target] = (True, timestamp)
                else:
                    events["pkp"][actor] = {
                        target: (True, timestamp)
                    }
                
        rounds_data[round_id] = events

    return rounds_data

In [ ]:
async def get_rounds_pdp_pkp_data_by_match_ids(match_ids: list[str]) -> dict:
    rounds_pdp_pkp_data_by_match_ids = {}
    for match_id in match_ids:
        rounds_pdp_pkp_data_by_match_id = await get_rounds_pdp_pkp_data_by_match_id(match_id)

        rounds_pdp_pkp_data_by_match_ids.update(rounds_pdp_pkp_data_by_match_id)

    return rounds_pdp_pkp_data_by_match_ids

In [ ]:
async def get_player_damage_kill_information_index_by_match_ids(match_ids: list[str]) -> pd.DataFrame:
    rounds_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)
    
    player_damage_kill_information_index_raw_data = {}
    for round_id, round_data in rounds_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                kill_damage_delta_t = []
                if actor_id in round_data["pdp"]:
                    if target_id in round_data["pdp"][actor_id]:
                        for damage_dealt, damage_timestamp in round_data["pdp"][actor_id][target_id]:
                            kill_damage_delta_t.append(
                                convert_str_timestamp_to_timestamp(kill_timestamp)-convert_str_timestamp_to_timestamp(damage_timestamp)
                            )
                
                if actor_id in player_damage_kill_information_index_raw_data:
                    player_damage_kill_information_index_raw_data[actor_id] += kill_damage_delta_t
                else:
                    player_damage_kill_information_index_raw_data[actor_id] = kill_damage_delta_t
    
    players_damage_kill_information_index = []
    for player, data in player_damage_kill_information_index_raw_data.items():
        mean_t_k_diff_t_d = np.mean(data)
        std_t_k_diff_t_d = np.std(data)
        player_damage_kill_information_index = std_t_k_diff_t_d / mean_t_k_diff_t_d
        
        players_damage_kill_information_index += [
            {
                "player": player,
                "mean_t_k_diff_t_d": mean_t_k_diff_t_d,
                "std_t_k_diff_t_d": std_t_k_diff_t_d,
                "player_damage_kill_information_index": player_damage_kill_information_index,
            }
        ]
    
    players_damage_kill_information_index = pd.DataFrame(players_damage_kill_information_index)
    
    return players_damage_kill_information_index

In [ ]:
async def get_player_kill_kill_information_index_by_match_ids(match_ids: list[str]) -> pd.DataFrame:
    rounds_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)

    player_kill_kill_information_index_raw_data = {}
    for round_id, round_data in rounds_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            kill_timestamps = []
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                kill_timestamps.append(kill_timestamp)
    
            if len(kill_timestamps) > 1:
                idxs = list(combinations(range(len(kill_timestamps)), 2))
    
                kill_timestamps_delta_t = []
                for (i, j) in idxs:
                    kill_timestamps_delta_t.append(
                        abs(convert_str_timestamp_to_timestamp(kill_timestamps[i])-convert_str_timestamp_to_timestamp(kill_timestamps[j]))
                    )
    
                if actor_id in player_kill_kill_information_index_raw_data:
                    player_kill_kill_information_index_raw_data[actor_id] += kill_timestamps_delta_t
                else:
                    player_kill_kill_information_index_raw_data[actor_id] = kill_timestamps_delta_t
            else:
                continue

    players_kill_kill_information_index = []
    for player, data in player_kill_kill_information_index_raw_data.items():
        mean_t_k_diff_t_k = np.mean(data)
        std_t_k_diff_t_k = np.std(data)
        player_kill_kill_information_index = std_t_k_diff_t_k / mean_t_k_diff_t_k
        
        players_kill_kill_information_index += [
            {
                "player": player,
                "mean_t_k_diff_t_k": mean_t_k_diff_t_k,
                "std_t_k_diff_t_k": std_t_k_diff_t_k,
                "player_kill_kill_information_index": player_kill_kill_information_index,
            }
        ]
    
    players_kill_kill_information_index = pd.DataFrame(players_kill_kill_information_index)
    
    return players_kill_kill_information_index

In [ ]:
match_ids = ["c5c7500c-6db8-45c8-b6ab-2be01b401ef7", "c7c65bec-8c54-4dea-afcd-c057320e1f8f"]

In [ ]:
players_damage_kill_information_index = await get_player_damage_kill_information_index_by_match_ids(match_ids)

players_damage_kill_information_index

In [ ]:
players_kill_kill_information_index = await get_player_kill_kill_information_index_by_match_ids(match_ids)

players_kill_kill_information_index

In [ ]:
rounds_pdp_pkp_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)

In [ ]:
player_id = "76561198031908632"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

    if player_id in rounds_pdp_pkp_data[round_id]["pdp"]:
        print("="*10, "Damages", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pdp"][player_id])

In [ ]:
player_id = "76561198070876270"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

    if player_id in rounds_pdp_pkp_data[round_id]["pdp"]:
        print("="*10, "Damages", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pdp"][player_id])

In [ ]:
player_id = "76561198030545997"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

In [ ]:
player_id = "76561198365697700"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])